## Import required packages

In [0]:
# Download required packages
!pip -q install gdown missingno torch

%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import torch
import torch.nn as nn
from typing import *
import datetime
import gdown

import tqdm as tq
def tqdm(*args, **kwargs):
  ''' Small trick to prevent tqdm printing newlines at each step. '''
  return tq.tqdm(*args, **kwargs, leave=True, position=0)

WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

## Data aquisition
We retrieve our datasets and download them to a temporary directory in the driver node.

In [0]:
gdown.download('https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM', '/tmp/data.zip', quiet=False)
!unzip -q /tmp/data.zip -d /tmp/
!rm /tmp/data.zip

Downloading...
From: https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM
To: /tmp/data.zip
0.00B [00:00, ?B/s]4.72MB [00:00, 23.6MB/s]15.2MB [00:00, 56.8MB/s]22.5MB [00:00, 48.1MB/s]28.3MB [00:00, 44.9MB/s]34.1MB [00:00, 37.1MB/s]50.9MB [00:01, 55.8MB/s]64.5MB [00:01, 72.8MB/s]71.9MB [00:01, 51.5MB/s]

Then we load the datasets to the DBFS.

In [0]:
dbutils.fs.mv("file:/tmp/data", "dbfs:/data", recurse=True)

Out[3]: True

In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


## Dataset loading

In [0]:
key_stats_df = spark.read.load("dbfs:/data/key_stats_yahoo.csv", 
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true"
                          )

# Drop the first ID column
key_stats_df = key_stats_df.drop(key_stats_df.columns[0])

# Use legacy format to parse dates
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
key_stats_df = key_stats_df.withColumn("Date", to_date(key_stats_df["Date"], 'MM/dd/yyyy HH:mm'))

# Cast numerical columns to double
for column in key_stats_df.columns[2:]:
  key_stats_df = key_stats_df.withColumn(column, key_stats_df[column].cast("double"))

# Prices dataframes for every stock
prices_files = [f.path for f in dbutils.fs.ls('/data/prices/') if f.path.endswith('.csv')]
dfs_names = [f.rsplit('/', 1)[1][:-len('.csv')] for f in prices_files]
prices_dfs = []
for f in tqdm(prices_files, desc='Reading stock price data', total=len(prices_files)):
  df = spark.read.load(f,
                       format="csv",
                       sep=",",
                       inferSchema="true",
                       header="true"
                      )
  df = df.withColumn("Date", to_date(df["Date"], 'dd-MM-yyyy'))
  prices_dfs.append(df)

Reading stock price data: 0%| | 0/429 [00:00<?, ?it/s]Reading stock price data: 0%| | 1/429 [00:01<07:26, 1.04s/it]Reading stock price data: 0%| | 2/429 [00:01<05:30, 1.29it/s]Reading stock price data: 1%| | 3/429 [00:02<05:22, 1.32it/s]Reading stock price data: 1%| | 4/429 [00:03<05:25, 1.31it/s]Reading stock price data: 1%| | 5/429 [00:03<04:59, 1.41it/s]Reading stock price data: 1%|▏ | 6/429 [00:04<05:09, 1.37it/s]Reading stock price data: 2%|▏ | 7/429 [00:05<05:21, 1.31it/s]Reading stock price data: 2%|▏ | 8/429 [00:05<05:02, 1.39it/s]Reading stock price data: 2%|▏ | 9/429 [00:06<04:46, 1.47it/s]Reading stock price data: 2%|▏ | 10/429 [00:07<04:54, 1.42it/s]Reading stock price data: 3%|▎ | 11/429 [00:08<04:52, 1.43it/s]Reading stock price data: 3%|▎ | 12/429 [00:08<04:48, 1.44it/s]Reading stock price data: 3%|▎ | 13/429 [00:09<04:38, 1.50it/s]Reading stock price data: 3%|▎ | 14/429 [00:10<04:52, 1.42it/s]Reading stock price data: 3%|▎ | 15/429 [00:10<04:52, 1.41it/s]Reading stock price data: 4%|▎ | 16/429 [00:11<04:42, 1.46it/s]Reading stock price data: 4%|▍ | 17/429 [00:12<04:35, 1.50it/s]Reading stock price data: 4%|▍ | 18/429 [00:12<04:25, 1.55it/s]Reading stock price data: 4%|▍ | 19/429 [00:13<04:20, 1.57it/s]Reading stock price data: 5%|▍ | 20/429 [00:13<04:13, 1.61it/s]Reading stock price data: 5%|▍ | 21/429 [00:14<05:02, 1.35it/s]Reading stock price data: 5%|▌ | 22/429 [00:15<05:18, 1.28it/s]Reading stock price data: 5%|▌ | 23/429 [00:16<05:00, 1.35it/s]Reading stock price data: 6%|▌ | 24/429 [00:17<05:34, 1.21it/s]Reading stock price data: 6%|▌ | 25/429 [00:18<05:14, 1.28it/s]Reading stock price data: 6%|▌ | 26/429 [00:18<04:58, 1.35it/s]Reading stock price data: 6%|▋ | 27/429 [00:19<04:51, 1.38it/s]Reading stock price data: 7%|▋ | 28/429 [00:20<04:40, 1.43it/s]Reading stock price data: 7%|▋ | 29/429 [00:20<04:35, 1.45it/s]Reading stock price data: 7%|▋ | 30/429 [00:21<04:43, 1.41it/s]Reading stock price data: 7%|▋ | 31/429 [00:22<04:47, 1.39it/s]Reading stock price data: 7%|▋ | 32/429 [00:22<04:34, 1.45it/s]Reading stock price data: 8%|▊ | 33/429 [00:23<04:26, 1.48it/s]Reading stock price data: 8%|▊ | 34/429 [00:24<04:24, 1.49it/s]Reading stock price data: 8%|▊ | 35/429 [00:24<04:22, 1.50it/s]Reading stock price data: 8%|▊ | 36/429 [00:25<04:17, 1.53it/s]Reading stock price data: 9%|▊ | 37/429 [00:26<04:16, 1.53it/s]Reading stock price data: 9%|▉ | 38/429 [00:26<04:14, 1.53it/s]Reading stock price data: 9%|▉ | 39/429 [00:27<04:13, 1.54it/s]Reading stock price data: 9%|▉ | 40/429 [00:27<04:04, 1.59it/s]Reading stock price data: 10%|▉ | 41/429 [00:28<04:04, 1.59it/s]Reading stock price data: 10%|▉ | 42/429 [00:29<04:06, 1.57it/s]Reading stock price data: 10%|█ | 43/429 [00:30<04:47, 1.34it/s]Reading stock price data: 10%|█ | 44/429 [00:31<04:50, 1.32it/s]Reading stock price data: 10%|█ | 45/429 [00:31<04:44, 1.35it/s]Reading stock price data: 11%|█ | 46/429 [00:32<04:28, 1.42it/s]Reading stock price data: 11%|█ | 47/429 [00:33<04:27, 1.43it/s]Reading stock price data: 11%|█ | 48/429 [00:33<04:24, 1.44it/s]Reading stock price data: 11%|█▏ | 49/429 [00:34<04:34, 1.38it/s]Reading stock price data: 12%|█▏ | 50/429 [00:35<04:22, 1.44it/s]Reading stock price data: 12%|█▏ | 51/429 [00:35<04:25, 1.42it/s]Reading stock price data: 12%|█▏ | 52/429 [00:36<04:49, 1.30it/s]Reading stock price data: 12%|█▏ | 53/429 [00:37<04:50, 1.29it/s]Reading stock price data: 13%|█▎ | 54/429 [00:38<04:31, 1.38it/s]Reading stock price data: 13%|█▎ | 55/429 [00:38<04:34, 1.36it/s]Reading stock price data: 13%|█▎ | 56/429 [00:39<04:48, 1.30it/s]Reading stock price data: 13%|█▎ | 57/429 [00:40<04:45, 1.30it/s]Reading stock price data: 14%|█▎ | 58/429 [00:41<04:33, 1.35it/s]Reading stock price data: 14%|█▍ | 59/429 [00:41<04:28, 1.38it/s]Reading stock price data: 14%|█▍ | 60/429 [00:42<04:22, 1.40it/s]Reading stock price data: 14%|█▍ | 61/429 [00:43<04:10, 1.47it/s]Reading stock price d

## Dataset analysis

In [0]:
print("Key stats dataframe format:")
prices_dfs[0].printSchema()

Key stats dataframe format:
root
-- Date: date (nullable = true)
-- Low: double (nullable = true)
-- Open: double (nullable = true)
-- Volume: integer (nullable = true)
-- High: double (nullable = true)
-- Close: double (nullable = true)
-- Adjusted Close: double (nullable = true)

In [0]:
print("Prices dataframe format:")
key_stats_df.printSchema()

Prices dataframe format:
root
-- Date: date (nullable = true)
-- Ticker: string (nullable = true)
-- Price: double (nullable = true)
-- DE Ratio: double (nullable = true)
-- Trailing P/E: double (nullable = true)
-- Price/Sales: double (nullable = true)
-- Price/Book: double (nullable = true)
-- Profit Margin: double (nullable = true)
-- Operating Margin: double (nullable = true)
-- Return on Assets: double (nullable = true)
-- Return on Equity: double (nullable = true)
-- Revenue Per Share: double (nullable = true)
-- Market Cap: double (nullable = true)
-- Enterprise Value: double (nullable = true)
-- Forward P/E: double (nullable = true)
-- PEG Ratio: double (nullable = true)
-- Enterprise Value/Revenue: double (nullable = true)
-- Enterprise Value/EBITDA: double (nullable = true)
-- Revenue: double (nullable = true)
-- Gross Profit: double (nullable = true)
-- EBITDA: double (nullable = true)
-- Net Income Avl to Common : double (nullable = true)
-- Diluted EPS: double (nullable = true)
-- Earnings Growth: double (nullable = true)
-- Revenue Growth: double (nullable = true)
-- Total Cash: double (nullable = true)
-- Total Cash Per Share: double (nullable = true)
-- Total Debt: double (nullable = true)
-- Current Ratio: double (nullable = true)
-- Book Value Per Share: double (nullable = true)
-- Cash Flow: double (nullable = true)
-- Beta: double (nullable = true)

### Utility functions

In [0]:
# TODO: add remaining utility functions

def missing_values_summary(df):
  ''' Returns a utility summary to view missing values in our dataframe. '''
  n = df.count()
  
  def to_percentage(x: pyspark.sql.column.Column, n: int) -> int:
    ''' Utility function to compute the amount of missing values as a percentage of the original dataframe. '''
    return round(100 * x / n, 3)
  
  # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe
  return df.agg(*[to_percentage(count(when(isnull(c), c)), n).alias(c) for c in df.columns]).first()

In [0]:
print("Overview of the missing values in the key_stats dataframe\n")
key_stats_summary = missing_values_summary(key_stats_df)
key_stats_summary

Overview of the missing values in the key_stats dataframe

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3309289877154145> in <module> 
 1 print ( "Overview of the missing values in the key_stats dataframe\n" ) 
 ----> 2 key_stats_summary = missing_values_summary ( key_stats_df ) 
 3 key_stats_summary

 <command-3309289877154152> in missing_values_summary (df) 
 10 
 11 # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe 
 ---> 12 return df . agg ( * [ to_percentage ( count ( when ( isnull ( c ) , c ) ) , n ) . alias ( c ) for c in df . columns ] ) . first ( ) 

 <command-3309289877154152> in <listcomp> (.0) 
 10 
 11 # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe 
 ---> 12 return df . agg ( * [ to_percentage ( count ( when ( isnull ( c ) , c ) ) , n ) . alias ( c ) for c in df . columns ] ) . first ( ) 

 TypeError : 'int' object is not callable

In [0]:
def prices_df_nan_summary(prices_dfs, names):
    ''' Utility function to summarize columns that have missing values. '''
    nan_dfs = []
    for i, (prices_df, name) in enumerate(zip(prices_dfs, names)):
        nan_cols = []
        nan_values = []
        for column in prices_df.columns:
            nan_absolute = prices_df.filter(prices_df[column].isNull()).count()
            if nan_absolute > 0:
                # Only consider columns that cointain NaN values
                nan_cols.append(column)
                nan_values.append(nan_absolute)
        if len(nan_cols):
            # Either we have all the data for a given day or we don't have any data for it
            assert len(set(nan_values)) == 1
            count = nan_values[0]
            nan_dfs.append((name, round(100*count/prices_df.count(), 3), count))
    return nan_dfs
    #return pd.DataFrame(nan_dfs, columns=['Stock name', 'Missing data (%)', 'Count'])

### Missing values imputation

In [0]:
''' TODO: fix prices_df_nan_summary '''

nan_dfs = []
for i, (prices_df, name) in enumerate(tqdm(zip(prices_dfs, dfs_names), total=len(prices_dfs))):
  nan_absolute = prices_df.agg(*[count(when(isnull(c), c)).alias(c) for c in prices_df.columns]).first()
  if any(nan_absolute):
    #count = nan_absolute.Close
    print(count)
    nan_dfs.append((name, round(100*count/prices_df.count(), 3), count))
  
  #[print(x) for x in nan_absolute]
  
  #if nan_absolute > 0:
    #print(nan_absolute)
    
  

0%| | 0/429 [00:00<?, ?it/s] 0%| | 0/429 [00:00<?, ?it/s]

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3309289877154160> in <module> 
 1 nan_dfs = [ ] 
 2 for i , ( prices_df , name ) in enumerate ( tqdm ( zip ( prices_dfs , dfs_names ) , total = len ( prices_dfs ) ) ) : 
 ----> 3 nan_absolute = prices_df . agg ( * [ count ( when ( isnull ( c ) , c ) ) . alias ( c ) for c in prices_df . columns ] ) . first ( ) 
 4 if any ( nan_absolute ) : 
 5 #count = nan_absolute.Close 

 <command-3309289877154160> in <listcomp> (.0) 
 1 nan_dfs = [ ] 
 2 for i , ( prices_df , name ) in enumerate ( tqdm ( zip ( prices_dfs , dfs_names ) , total = len ( prices_dfs ) ) ) : 
 ----> 3 nan_absolute = prices_df . agg ( * [ count ( when ( isnull ( c ) , c ) ) . alias ( c ) for c in prices_df . columns ] ) . first ( ) 
 4 if any ( nan_absolute ) : 
 5 #count = nan_absolute.Close 

 TypeError : 'int' object is not callable

In [0]:
print(prices_dfs[0].columns)
prices_dfs[0].filter(prices_dfs[0]['Date'].isNull()).count()

['Date', 'Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close']
Out[135]: 0